In [117]:
%matplotlib inline
from mxnet import np, npx, autograd
from mxnet.gluon import nn, Trainer
from mxnet.gluon.loss import L2Loss
import mxnet.initializer as initializer
from d2l import mxnet as d2l
from PIL import Image
from pdb import set_trace
npx.set_np()

In [141]:
X = np.zeros((6, 9))
X[:, 0:2] = 1
X[:, -1:-3:-1] = 1

In [142]:
def crosscorr2d(X, K):
    n_x, n_y = X.shape
    k_x, k_y = K.shape
    result = []
    result_xdim = (n_x - k_x + 1)
    result_ydim = (n_y - k_y + 1)
    Y = np.zeros(shape=((n_x - k_x + 1), (n_y - k_y + 1)))
    for i in range(result_xdim):
        for j in range(result_ydim):
            result.append((X[i:(i + k_x), j:(j + k_y)]*K).sum())
    Y = np.array(result).reshape((result_xdim, result_ydim))
    #Y = np.repeat(np.repeat(K, 4, axis=1), 6, axis=0)
    return Y

In [143]:
K = np.array([[1, -1]])
y = crosscorr2d(X, K)

In [144]:
X = X.reshape(1, 1, X.shape[0], X.shape[1])
y = y.reshape(1, 1, y.shape[0], y.shape[1])

In [113]:
class CustomConv2D(nn.Block):
    def __init__(self, kernel_shape, **kwargs):
        super().__init__(**kwargs)
        self.kernel = self.params.get('kernel', shape=kernel_shape)
    
    def forward(self, x):
        yhat = crosscorr2d(x, self.kernel.data())
        return yhat

In [125]:
net = nn.Sequential()
net.add(nn.Conv2D(1, (1, 2)))
net.initialize()

In [129]:
num_epochs = 100
lr = 0.1
optim = Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
for epoch in range(num_epochs):
    with autograd.record():
        yhat = net(X)
        loss = L2Loss()(yhat, y)
    loss.backward()
    optim.step(batch_size=1)

In [130]:
net[0].weight.data()

array([[[[ 0.9140745 , -0.91406244]]]])

In [145]:
X1 = np.repeat(X, 3, axis=1).squeeze()

In [150]:
K1 = np.repeat(K[np.newaxis, : , :], 3, axis=0)

In [166]:
X1.shape

(3, 6, 9)

In [155]:
next(abc)

(array([[1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1.]]),
 array([[ 1., -1.]]))

In [182]:
from functools import reduce
def corr2d_multi_in(X, K):
    channel_corr2d = [crosscorr2d(channel, channel_kernel) for channel, channel_kernel in zip(X, K)]
    return sum(channel_corr2d)

In [183]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of `K`, and each time, perform
    # cross-correlation operations with input `X`. All of the results are
    # stacked together
    return np.stack([corr2d_multi_in(X, k) for k in K], 0)

In [189]:
K2 = np.repeat(K1[np.newaxis], 5, axis=0)

In [191]:
corr2d_multi_in_out(X1, K2).shape

(5, 6, 8)

In [181]:
multicorr_in(X1, K1)

array([[ 0.,  3.,  0.,  0.,  0.,  0., -3.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0., -3.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0., -3.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0., -3.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0., -3.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0., -3.,  0.]])

## Appendix

In [ ]:
def crosscorr2d_old(X, K):
    n_x, n_y = X.shape
    k_x, k_y = K.shape
    Y = np.zeros(shape=((n_x - k_x + 1), (n_y - k_y + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:(i + k_x), j:(j + k_y)]*K).sum()
    return Y